<a href="https://colab.research.google.com/github/somaSystems/hyperThelia/blob/main/Run_Demo_hyperthelia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###UNCOMMENT AND MOUNT DRIVE FOR MULTISESSION USE AND SAVING###

# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# 1. Clone if needed + bootstrap
import sys
from pathlib import Path


REPO_URL = "https://github.com/somaSystems/HyperThelia.git"
CLONE_PARENT_DIR = Path("/content")
CLONE_DIR = CLONE_PARENT_DIR / "HyperThelia"
BASE_PROJECT_DIR = CLONE_DIR / "Hyperthelia_project"
LIB_DIR = BASE_PROJECT_DIR / "notebooks" / "lib"

if not CLONE_DIR.exists():
    print("Cloning HyperThelia repo...")
    !git clone {REPO_URL} {CLONE_DIR}
else:
    print("HyperThelia repo already exists.")

if str(LIB_DIR) not in sys.path:
    sys.path.insert(0, str(LIB_DIR))

# 2. Now safe to import functions
from setup_functions import clone_hyperthelia_repo, setup_project_io

# 3. Use functions
CLONE_DIR, BASE_PROJECT_DIR = clone_hyperthelia_repo(CLONE_PARENT_DIR)


In [ ]:
from pathlib import Path

# === BASE PROJECT SETUP ===
BASE_PROJECT_DIR = Path("/content/HyperThelia/Hyperthelia_project")
RAW_DIR = Path("/content/HyperThelia/Hyperthelia_project/data_demo")



In [ ]:
# Install latest Cellpose from GitHub (includes SAM support)
!pip install git+https://github.com/mouseland/cellpose.git

In [ ]:
# # === SESSION SETUP IMPORTS AND FUNCTIONS ===
import segmentation
import importlib
importlib.reload(segmentation)

RAW_DIR, OUTPUTS_DIR = setup_project_io(BASE_PROJECT_DIR)

# SETUP LOGGER, CHECK GPU, and CELLPOSE MODEL

from segmentation import setup_cellpose_model
model = setup_cellpose_model(gpu=True)


In [ ]:
from cellpose import models, core, io

io.logger_setup()  # Enables progress printing

# Check if GPU is available
if not core.use_gpu():
    raise ImportError("No GPU access. Change your Colab runtime to GPU.")

# Load Cellpose model (SAM backend)
model = models.CellposeModel(gpu=True)

In [ ]:
# === LOOK FOR EXPERIMENTS IN RAW_DIR ===
from segmentation import print_experiment_summary
print_experiment_summary(RAW_DIR)


In [ ]:
from segmentation import run_segmentation_pipeline

# === SEGMENTATION PARAMETERS ===
Z_AXIS = 0
CHANNEL_AXIS = 1
BATCH_SIZE = 32
DO_3D = False
STITCH_THRESHOLD = 0.5

# === RUN PIPELINE ===
run_segmentation_pipeline(
    RAW_DIR,
    OUTPUTS_DIR,
    model,
    z_axis=Z_AXIS,
    channel_axis=CHANNEL_AXIS,
    batch_size=BATCH_SIZE,
    do_3D=DO_3D,
    stitch_threshold=STITCH_THRESHOLD
)


In [ ]:
from measurevisualise import interactive_segmentation_viewer
interactive_segmentation_viewer(BASE_PROJECT_DIR / "outputs")


PART II TRACKING

In [ ]:
import tracking
import propagation
import visualisation

import importlib
# importlib.reload(tracking)
# importlib.reload(propagation)
# importlib.reload(visualisation)

In [ ]:
# === AUTO CONFIGURATION ===

# Tracking parameters (editable by user)
XY_UM = 0.325
Z_UM = 1.0
MAX_DIST_UM = 10
MIN_VOLUME = 200
MAX_VOLUME = 8000
EDGE_MARGIN = 1
TRACKING_MODE = "nearest"

In [ ]:
# === STEP 1: VISUALISE PRE-TRACKING ===
from visualisation import plot_volume_histogram_for_experiment, view_segmentation_slice_with_boundaries, get_segmented_tiffs_by_experiment

experiments = get_segmented_tiffs_by_experiment(OUTPUTS_DIR)

plot_volume_histogram_for_experiment(exp_index=0,
                                                   experiments_dict=experiments,
                                                   min_volume=MIN_VOLUME,
                                                   max_volume=MAX_VOLUME)

view_segmentation_slice_with_boundaries(exp_index=0,
                                        experiments_dict=experiments,
                                        time_index=0,
                                        z_slice=18,
                                        edge_margin=EDGE_MARGIN)



In [ ]:
# === STEP 2: RUN TRACKING ===
tracking.run_tracking_pipeline(
    output_base_dir=OUTPUTS_DIR,
    xy_um=XY_UM,
    z_um=Z_UM,
    max_dist_um=MAX_DIST_UM,
    min_volume=MIN_VOLUME,
    max_volume=MAX_VOLUME,
    edge_margin=EDGE_MARGIN,
    tracking_mode=TRACKING_MODE
)

In [ ]:
# === STEP 3: PROPAGATE LABELS ===
propagation.run_propagation_pipeline(output_base_dir=OUTPUTS_DIR)

In [ ]:
# === STEP 4: VISUALISE POST-TRACKING ===
visualisation.plot_tracked_centroids_xy_by_index(exp_index=0, output_base_dir=OUTPUTS_DIR)
visualisation.preview_propagated_labels_zslice(exp_index=0, z_slice=18, output_base_dir=OUTPUTS_DIR)

Part III: Measurement

In [ ]:
import importlib
import measurement
import measurevisualise
importlib.reload(measurevisualise)
importlib.reload(measurement)

In [ ]:
# ===  USER TOGGLES FOR MEASURES ===
is_tracked = True
compute_surface = True
enable_intensity_measurement = False
intensity_channel_mode = "folder"
force = False
RAW_INTENSITY_DIR = BASE_PROJECT_DIR / "raw_intensity"
EXPORT_DIR = BASE_PROJECT_DIR / "image_exports"


In [ ]:
# ===  LIST AVAILABLE EXPERIMENTS ===
experiment_data = measurement.discover_experiments(OUTPUTS_DIR, is_tracked=is_tracked)
measurement.summarise_experiment_data(experiment_data)

In [ ]:
# ===  MEASURE EXPERIMENTS ===
measurement.run_all_measurements(
    experiment_data=experiment_data,
    is_tracked=is_tracked,
    compute_surface=compute_surface,
    enable_intensity_measurement=enable_intensity_measurement,
    intensity_dir=RAW_INTENSITY_DIR,
    force=force
)


In [ ]:
# ===  LIST AVAILABLE MEASUREMENT CSVs ===
measurevisualise.list_available_measurement_csvs(BASE_PROJECT_DIR)


In [ ]:
import pandas as pd

# Load the CSV and show all available measurement columns
csv_path = BASE_PROJECT_DIR / "outputs/outputs_demothelia/measured/regionprops_demothelia_tracked.csv"
df = pd.read_csv(csv_path)

print("Available measurement columns:")
print(df.columns.tolist())


In [ ]:

# ===  CHOOSE ONE MEASUREMENT CSV TO EXPLORE ===
csv_path = BASE_PROJECT_DIR / "outputs/outputs_demothelia/measured/regionprops_demothelia_tracked.csv"

# ===  VIEW A MEASUREMENT OVERLAY ===
timepoint = 0
z_slice = 19
value_column = "area_voxels"  # or e.g. "intensity_mean_C1", "elongation", etc.

measurevisualise.view_by_csv(
    csv_path=csv_path,
    base_dir=BASE_PROJECT_DIR,
    timepoint=timepoint,
    z=z_slice,
    value_column=value_column
)

In [ ]:
# === Export TIFF ===
measurevisualise.export_measurement_values_as_tiff(csv_path,
                                  BASE_PROJECT_DIR,
                                  timepoint = timepoint,
                                  value_column = value_column,
                                  output_dir=BASE_PROJECT_DIR / "image_exports")


